In [54]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

from typing import List

import os
import json
import random

# Data

In [55]:
PATH = "..\\src\\data\\komus\\dataset.json"

with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

len(data)

83755

# Utils

In [56]:
def create_model_prompts(system_prompt: str,
                         user_prompt: str) -> ChatPromptTemplate:
    system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_prompt,
         user_prompt]
    )
    return chat_prompt

# Models

## Gemini

In [57]:
PROJECT_ID = 'axial-chemist-425510-p2'
LOCATION = "europe-west4"

# Укажите путь к вашему JSON-файлу с кредами
SERVICE_ACCOUNT_FILE = "../secrets/axial-chemist-425510-p2-b4eb1d622fbe.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_FILE


llm_params = {
    'model_name': 'gemini-2.0-flash-001',
    'project': PROJECT_ID,
    'location': LOCATION
}

llm = ChatVertexAI(**llm_params)

## Т-Pro

In [58]:
from langchain_openai import ChatOpenAI

T_PRO_CREDS = "../secrets/t-pro.json"

with open(T_PRO_CREDS) as file:
    model_params = json.load(file)

llm = ChatOpenAI(**model_params)

## Случайный выбор из подборки

In [59]:
idx = random.choices(list(range(len(data))), k = 10)
idx = [22009, 6842, 3491, 11635, 54735, 66683, 64314, 31085, 17752, 8034]

product_choices = [data[id] for id in idx]

# Аугментация

In [60]:
"""
The output should consist of the product item that was augmented.

Pay attention to the strict observance of these rules.:
- No data from the product name ** should be lost**.
- No data should be added to the result.
"""

SYSTEM_PROMPT = """
Проведи аугментацию товарной позиции по данным примерам.
Examples:
{examples}

**Не создавай больше текста, кроме ответа**
""".strip()

USER_PROMPT = """Проведи аугментацию для данной позиции {problem_title}"""

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [61]:
examples = [
    {
        "original_item": "4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota",
        "clean_item": "Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)"
    },
    {
        "original_item": "АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT",
        "clean_item": "Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт."
    },
        {
        "original_item": "Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют",
        "clean_item": "Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют "
    },
    {
        "original_item": "Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт",
        "clean_item": "Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)"
    }
]

examples_str = "\n".join([f"""
{i+1})Оригинальная товарная позиция: {product["clean_item"]}
Аугментированная товарная позиция: {product['original_item']}
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Оригинальная товарная позиция: Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)
Аугментированная товарная позиция: 4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota
2)Оригинальная товарная позиция: Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт.
Аугментированная товарная позиция: АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT
3)Оригинальная товарная позиция: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют 
Аугментированная товарная позиция: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют
4)Оригинальная товарная позиция: Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)
Аугментированная товарная позиция: Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт


## Запуск модели

In [62]:
batch = [
    {
        "examples": examples_str,
        "problem_title": data[product_id]["title"]
    } for product_id in idx
]
batch

[{'examples': '1)Оригинальная товарная позиция: Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)\nАугментированная товарная позиция: 4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota\n2)Оригинальная товарная позиция: Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт.\nАугментированная товарная позиция: АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT\n3)Оригинальная товарная позиция: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют \nАугментированная товарная позиция: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют\n4)Оригинальная товарная позиция: Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)\nАугментированная товарная позиция: Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт',
  'problem_title': 'Картридж лазерный Хеrох 106R01277 черный оригинальный (двойная упаковка)'},
 {'examples': '1)Оригинальная товарная позиция:

In [63]:
augment_chain = prompt | llm

res1 = augment_chain.batch(batch)
res2 = augment_chain.batch(batch)

In [64]:
from difflib import ndiff

def highlight_diff(s1, s2):
    diff = ndiff(s1, s2)
    markdown_diff = ""
    
    for part in diff:
        if part.startswith('+'):
            markdown_diff += f'<span style="color: green;">{part[2:]}</span>'  # Добавленное
        elif part.startswith('-'):
            markdown_diff += f'<span style="color: red; text-decoration: line-through;">{part[2:]}</span>'  # Удалённое
        else:
            markdown_diff += part[2:]  # Общий текст
    
    return markdown_diff

In [65]:
print("\n\n".join(
        ["\n\n".join(
            ["\\\n".join([title, augmented_title.content, highlight_diff(title, augmented_title.content)]) for augmented_title in augmented_titles]
        ) for title, *augmented_titles in zip([data[id]["title"] for id in idx], res1, res2)]
    )
)

Картридж лазерный Хеrох 106R01277 черный оригинальный (двойная упаковка)\
Картридж лазерный 106R01277 черный ORIG. ХЕРОХ 2УПАК\
Картридж лазерный <span style="color: red; text-decoration: line-through;">Х</span><span style="color: red; text-decoration: line-through;">е</span><span style="color: red; text-decoration: line-through;">r</span><span style="color: red; text-decoration: line-through;">о</span><span style="color: red; text-decoration: line-through;">х</span><span style="color: red; text-decoration: line-through;"> </span>106R01277 черный <span style="color: green;">O</span><span style="color: green;">R</span><span style="color: green;">I</span><span style="color: green;">G</span><span style="color: green;">.</span><span style="color: red; text-decoration: line-through;">о</span><span style="color: red; text-decoration: line-through;">р</span><span style="color: red; text-decoration: line-through;">и</span><span style="color: red; text-decoration: line-through;">г</span><span s

# Сравнение двух товарных позиций

In [75]:
class CompareListResponse(BaseModel):
    lost_fragments: List[str] = Field(..., description="Фрагменты строки, которые были потеряны")
    added_fragments: List[str] = Field(..., description="Фрагменты, которые были добавлены в аугментированную строку")

parser = PydanticOutputParser(pydantic_object=CompareListResponse)

SYSTEM_PROMPT = """
Дана оригинальная строка str1 и аугментированная строка str2. Необходимо:  
1. Найти фрагменты из str1, которые потерялись при аугментации (отсутствуют в str2).  
2. Найти фрагменты из str2, которые добавились при аугментации (отсутствуют в str1).    

Примеры:
{examples}

Правила:
- Если фрагмент сокращен при аугментации, то он не потерян.

В ходе рассуждения напиши ответ только один раз.

{format_instructions}
""".strip()

USER_PROMPT = """
Оригинальная строка: {original_string}
Аугментированная строка: {augmented_string}

Сравни две эти строки.
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

## Примеры

In [69]:
examples = [
    {
        "original_string": "Батарея ExeGate HR 12-9 12 Вольт 9 Ач (EP129860RUS)",
        "augmented_string": "БАТАРЕЯ ДЛЯ ИБП EXEGATE HR 12-9, 12В",
        "lost": ["9 Ач", "(EP129860RUS)"],
        "added": ["ДЛЯ ИБП"]
    },
    {
        "original_string": "Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют",
        "augmented_string": "Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют",
        "lost": [],
        "added": []
    },
    {
        "original_string": "Поддон деревянный, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)",
        "augmented_string": "Поддон деревянный 1500 кг БУ 1200х800мм",
        "lost": ["грузоподъемность", "1 сорт"],
        "added": ["БУ"]
    },
    {
        "original_string": "Картридж лазерный Хеrох 106R01277 оригинальный (двойная упаковка)",
        "augmented_string": "Картридж лазерный HEROCH 106R01277 черный ориг. ДУ",
        "lost": ["Хеrох"],
        "added": ["HEROCH", "черный"]
    }
]

examples_str = "\n".join([f"""
{i+1})
Оригинальная строка: {example["original_string"]}
Аугментированная строка: {example["augmented_string"]}
Ответ: {json.dumps({
    "lost_fragments": example["lost"],
    "added_fragments": example["added"]
}, ensure_ascii=False)}
""".strip() for i, example in enumerate(examples)])
print(examples_str)

1)
Оригинальная строка: Батарея ExeGate HR 12-9 12 Вольт 9 Ач (EP129860RUS)
Аугментированная строка: БАТАРЕЯ ДЛЯ ИБП EXEGATE HR 12-9, 12В
Ответ: {"lost_fragments": ["9 Ач", "(EP129860RUS)"], "added_fragments": ["ДЛЯ ИБП"]}
2)
Оригинальная строка: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют
Аугментированная строка: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют
Ответ: {"lost_fragments": [], "added_fragments": []}
3)
Оригинальная строка: Поддон деревянный, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)
Аугментированная строка: Поддон деревянный 1500 кг БУ 1200х800мм
Ответ: {"lost_fragments": ["грузоподъемность", "1 сорт"], "added_fragments": ["БУ"]}
4)
Оригинальная строка: Картридж лазерный Хеrох 106R01277 оригинальный (двойная упаковка)
Аугментированная строка: Картридж лазерный HEROCH 106R01277 черный ориг. ДУ
Ответ: {"lost_fragments": ["Хеrох"], "added_

In [71]:
batch = [{
    "examples": examples_str,
    "format_instructions": parser.get_format_instructions(),
    "original_string": product["title"],
    "augmented_string": res.content
} for product, res in zip(product_choices, res1)]
print(*batch, sep="\n")

{'examples': '1)\nОригинальная строка: Батарея ExeGate HR 12-9 12 Вольт 9 Ач (EP129860RUS)\nАугментированная строка: БАТАРЕЯ ДЛЯ ИБП EXEGATE HR 12-9, 12В\nОтвет: {"lost_fragments": ["9 Ач", "(EP129860RUS)"], "added_fragments": ["ДЛЯ ИБП"]}\n2)\nОригинальная строка: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют\nАугментированная строка: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют\nОтвет: {"lost_fragments": [], "added_fragments": []}\n3)\nОригинальная строка: Поддон деревянный, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)\nАугментированная строка: Поддон деревянный 1500 кг БУ 1200х800мм\nОтвет: {"lost_fragments": ["грузоподъемность", "1 сорт"], "added_fragments": ["БУ"]}\n4)\nОригинальная строка: Картридж лазерный Хеrох 106R01277 оригинальный (двойная упаковка)\nАугментированная строка: Картридж лазерный HEROCH 106R01277 черный ориг. ДУ\nОтвет: {"lost_f

In [77]:
compare_chain = prompt | llm

compare_results = compare_chain.batch(batch)

In [78]:
for item, compare_result in zip(batch, compare_results):
    print(item["original_string"])
    print(item["augmented_string"])
    print(compare_result.content)
    print(parser.invoke(compare_result))
    print("-------------------------------")
    

Картридж лазерный Хеrох 106R01277 черный оригинальный (двойная упаковка)
Картридж лазерный 106R01277 черный ORIG. ХЕРОХ 2УПАК
Для анализа двух строк:

**Оригинальная строка:** Картридж лазерный Хеrох 106R01277 черный оригинальный (двойная упаковка)  
**Аугментированная строка:** Картридж лазерный 106R01277 черный ORIG. ХЕРОХ 2УПАК  

Мы можем определить следующие изменения:

1. "Хеrох" заменено на "ХЕРОХ". Этот переход считаю допустимым изменением, так как фрагмент не был утерян.
2. "оригинальный" сокращено до "ORIG.". Согласно правилам, это считается допустимой сокращенной версией фрагмента.
3. "двойная упаковка" сокращено до "2УПАК". Это также считается допустимым сокращением.
4. Фрагмент "106R01277" присутствует в обеих строках, но в аугментированной строке он предшествует "черный", чего не было в оригинале. Это изменение позиции, но не утеря или добавление фрагмента.
5. Отсутствует часть "лазерный" сразу после "Картридж". Однако, в данном примере "лазерный" присутствует в аугментир